# Analyzing Selfplay Data

In [80]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
from aegomoku.game_data import read_training_data
from aegomoku.utils import analyse_example

In [ ]:
BOARD_SIZE=15
filename = "/Users/wgiersche/workspace/Project-Ellie/DATA/00001.pickle"
filename = "/Users/wgiersche/workspace/Project-Ellie/DeepGomoku/notebooks/temp/00001.pickle"
examples = read_training_data(filename, BOARD_SIZE)
len(examples), len(examples[0]), examples[0][0].shape

---
### Data Verification
We have records from those games in 8 symmetries. Now we skim through a good chunk of training records to check whether it makes sense (to some degree).

In [ ]:
import random

analyse_example(BOARD_SIZE, examples[random.randint(0, len(examples))], policy_cutoff=.01)